In [1]:
# 修复依赖冲突
# 安装并导入 - 锁定兼容版本避免 Kaggle 环境依赖冲突
!pip install -q tensorboardX albumentations thop
# ===== 修复 Kaggle 2025 环境兼容性问题 =====
# 问题：Kaggle 升级到 NumPy 2.x 导致 SciPy/Seaborn/OpenCV 崩溃
# 解决：锁定到 2024 年稳定的版本组合

# 首先，按照原意圖解除安裝衝突的套件
print("正在清理衝突的包...")
!pip uninstall -y numpy scipy opencv-python opencv-contrib-python opencv-python-headless 2>/dev/null || true

# 接著，在單一指令中安裝所有必要的函式庫
# 這能讓 pip 一次性解決所有依賴關係，並遵循版本鎖定
print("\n安裝兼容版本的核心庫...")
!pip install --quiet \
    numpy==1.26.4 \
    scipy==1.11.4 \
    matplotlib==3.7.5 \
    opencv-python==4.8.1.78 \
    albumentations \
    tensorboardX \
    thop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import cv2
import glob
import time
import timm
import torch
import random
import logging
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt
import torch.nn.functional as F




from torch import nn
from PIL import Image
from tqdm import tqdm
from io import BytesIO
from thop import profile
from copy import deepcopy
from torch.optim import Adam
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.transforms import v2
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import default_collate 
from albumentations.pytorch import ToTensorV2
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
# ...
# 设置日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
class KaggleConfig:
    def __init__(self):
        # --- 路径设置 (请根据您的Kaggle数据集名称修改) ---
        # 假设您的Kaggle数据集地址是 /kaggle/input/your-dataset-name
        # 您需要将下面的 'your-dataset-slug' 替换为您的数据集的实际名称
        
        
        self.train_path = '/kaggle/input/data-base/Dataset_base_split_new/train'
        self.val_path = '/kaggle/input/data-base/Dataset_base_split_newval'
        self.test_path = '/kaggle/input/data-base-test/test'
        self.save_path = '/kaggle/working/' # 模型和输出文件将保存在这里

        # --- 训练参数 ---
        self.train_batch_size = 64
        self.val_batch_size = 64
        self.test_batch_size = 128
        self.num_epochs = 200
        self.cuda_no = 0
        
        # --- K折交叉验证参数 ---
        self.n_folds = 5  # K折交叉验证的折数
        self.current_fold = 0  # 当前折索引
        
        # 🆕 控制运行哪些fold
        self.start_fold = 4  # 从第4折开始（索引从0开始，所以4表示第5折，即最后一折）
        self.end_fold = 5    # 到第5折结束（不包含，所以只运行fold 4）
        
        # --- 模型和优化器参数 ---
        self.encoder = 'tf_efficientnetv2_b0'  # 🔥 修正为 resnet50（与您的需求一致）
        self.classnum = 8        # 您的总类别数
        self.LR_clf = 1e-5       # 🔥 修改为 5e-5
        self.LR_head = 1e-3     # <-- 新增：用于阶段一的学习率，要大得多
        self.WE_dec = 0.06
        self.min_lr = 1e-6

        #学习率策略
        # ReduceLROnPlateau 参数（仅在 lr_scheduler == "plateau" 时生效）
        self.lr_scheduler = "plateau"  # 想用余弦退火就改为 "cosine"
        self.plateau_factor = 0.5   # 每次降 LR 的倍数
        self.plateau_patience = 5   # 连续多少个 epoch 验证损失不下降才降 LR
        # 余弦的 T_max 设置（可按阶段分别设置）
        self.stage1_tmax = 15
        
        # --- 新增参数 ---
        self.use_hybrid_mix = 0 
        self.aug_methon = 1
        self.label_smoothing = 0.00 # 标签平滑的强度，0.1是一个常用值
        self.focal_loss_gamma = 2.0  # 添加 gamma 参数，2.0 是一个很好的起始值
        self.fold = 0
        self.dropout_rate = 0.0


In [4]:
class FocalLoss(nn.Module):
    """
    Focal Loss 的 PyTorch 实现。
    这个损失函数被设计用来解决类别不平衡和难易样本不平衡的问题。
    它通过一个动态缩放因子来降低“简单”样本对损失的贡献，
    从而让模型更加专注于学习那些“困难”的样本。

    公式: FL(p_t) = -α_t * (1 - p_t)^γ * log(p_t)
    """

    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        初始化 Focal Loss.
        Args:
            alpha (Tensor, optional): 一个与类别数量相同长度的张量，用于为每个类别分配权重。
                                      这与加权交叉熵中的 'weight' 参数作用相同。
                                      默认值: None (所有类别权重为1)。
            gamma (float, optional): 聚焦参数 (Focusing Parameter)。
                                     该值越大，模型就越会专注于困难样本。
                                     论文中推荐的默认值为 2.0。
            reduction (str, optional): 指定如何对批次的损失进行聚合。
                                       可选值: 'mean', 'sum', 'none'。
                                       默认值: 'mean'。
        """
        super(FocalLoss, self).__init__()
        # alpha 参数用于平衡类别重要性，可以是 None 或者一个 Tensor
        # 如果是 Tensor，我们会确保它在正确的设备上
        if alpha is not None:
            if not isinstance(alpha, torch.Tensor):
                alpha = torch.tensor(alpha)
        self.alpha = alpha

        # gamma 是聚焦参数，用于调节难易样本的权重
        self.gamma = gamma

        # reduction 指定最终损失的计算方式
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        计算 Focal Loss 的前向传播。
        Args:
            inputs (Tensor): 模型的原始输出 (logits)，形状为 [N, C]，
                             其中 N 是批次大小, C 是类别数。
            targets (Tensor): 真实的类别标签，形状为 [N]。
        Returns:
            Tensor: 计算出的 Focal Loss。
        """
        # 确保 alpha 权重张量和输入在同一个设备上 (CPU 或 GPU)
        if self.alpha is not None and self.alpha.device != inputs.device:
            self.alpha = self.alpha.to(inputs.device)

        # 步骤 1: 计算标准的交叉熵损失，但不进行任何聚合 (reduction='none')
        #         这样我们可以得到批次中每个样本的独立损失值。
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        # 步骤 2: 从交叉熵损失中计算出模型对正确类别的预测概率 p_t
        #         公式: p_t = exp(-ce_loss)
        #         这个技巧可以避免直接操作 softmax 的输出，数值上更稳定。
        pt = torch.exp(-ce_loss)

        # 步骤 3: 计算 Focal Loss 的核心部分——动态缩放因子
        #         公式: (1 - p_t)^γ
        #         这个因子会根据预测概率 pt 的大小来动态调整损失的权重。
        #         如果 pt 很大 (接近1，简单样本)，这个因子就接近0，损失被大幅削弱。
        #         如果 pt 很小 (接近0，困难样本)，这个因子就接近1，损失基本保持不变。
        focal_term = (1 - pt) ** self.gamma

        # 步骤 4: 计算最终的 Focal Loss
        #         首先将动态缩放因子与原始的交叉熵损失相乘。
        loss = focal_term * ce_loss

        # 步骤 5: (可选) 如果提供了 alpha 权重，则再乘上对应类别的权重
        if self.alpha is not None:
            # self.alpha[targets] 会根据 targets 中的标签索引，
            # 从 alpha 权重张量中取出每个样本对应的权重。
            alpha_t = self.alpha[targets]
            loss = alpha_t * loss

        # 步骤 6: 根据指定的 reduction 参数对最终的损失进行聚合
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


In [5]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
def count_parameters(model):
    """Counts the number of trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
def mean_and_std(paths):
    print('Calculating mean and std of training set for data normalization.')
    bgr_means, bgr_stds = [], []

    for img_path in tqdm(paths):
        img = cv2.imread(img_path)
        channel_means, channel_stds = cv2.meanStdDev(img)[:2]
        bgr_means.append(channel_means.reshape(3))
        bgr_stds.append(channel_stds.reshape(3))

    global_mean = np.mean(bgr_means, axis=0)[::-1] / 255.0
    global_std = np.mean(bgr_stds, axis=0)[::-1] / 255.0

    print(f"Mean (RGB): {global_mean.tolist()}")
    print(f"Std  (RGB): {global_std.tolist()}")
    return global_mean, global_std

def save_metrics_to_excel(file_name, epoch, encoder_lr=None, loss=None, accuracy=None,
                         precision=None, recall=None, f1=None, f1_macro=None, auc_macro=None,
                         auc_weighted=None, confusion_matrix=None, **kwargs):
    # 🆕 encoder_lr 作为第一列，用于追踪实验配置
    data = {
        'encoder_lr': [encoder_lr],  # 🆕 新增: 记录 run_id (例如 "convnext_tiny_lr_3e-5")
        'Epoch': [epoch], 
        'Loss': [loss], 
        'Accuracy': [accuracy], 
        'Precision': [precision],
        'Recall': [recall], 
        'F1_weighted': [f1], 
        'F1_macro': [f1_macro], 
        'AUC_macro': [auc_macro], 
        'AUC_weighted': [auc_weighted]
    }
    df = pd.DataFrame(data)
    
    # 在Kaggle环境中，直接使用save_path
    args = KaggleConfig()
    file_path = os.path.join(args.save_path, file_name)

    if not os.path.exists(file_path):
        df.to_excel(file_path, index=False, sheet_name='Sheet1')
    else:
        with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            if 'Sheet1' in writer.sheets:
                 start_row = writer.sheets['Sheet1'].max_row
                 df.to_excel(writer, sheet_name='Sheet1', index=False, header=False, startrow=start_row)
            else:
                 df.to_excel(writer, sheet_name='Sheet1', index=False)

class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def cm_to_figure(cm):
    fig = plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix')
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    image = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image.transpose(2, 0, 1)

def plot_history_curves(history, save_dir):
    """绘制并保存训练和验证的损失与准确率曲线图。"""
    # 检查history字典中是否包含所需的所有键
    required_keys = ['train_loss', 'val_loss', 'train_acc', 'val_acc']
    if not all(key in history for key in required_keys):
        print("Warning: History dictionary is missing one or more required keys for plotting. Skipping plot.")
        return

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle('Training & Validation History', fontsize=16)

    # 确保所有列表长度一致，以最短的为准
    min_epochs = len(history['train_loss'])
    epochs_range = range(min_epochs)

    # 绘制损失曲线
    ax1.plot(epochs_range, history['train_loss'][:min_epochs], 'o-', label='Train Loss')
    ax1.plot(epochs_range, history['val_loss'][:min_epochs], 'o-', label='Validation Loss')
    ax1.legend(loc='upper right')
    ax1.set_title('Loss vs. Epochs')
    ax1.set_ylabel('Loss')
    ax1.grid(True)

    # 绘制准确率曲线
    ax2.plot(epochs_range, history['train_acc'][:min_epochs], 'o-', label='Train Accuracy')
    ax2.plot(epochs_range, history['val_acc'][:min_epochs], 'o-', label='Validation Accuracy')
    ax2.legend(loc='lower right')
    ax2.set_title('Accuracy vs. Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])

    # 保存图片
    save_path = os.path.join(save_dir, 'history_curves.png')
    plt.savefig(save_path, dpi=300)
    plt.close(fig)
    print(f"✅ Training history curves plot saved to {save_path}")

In [6]:
# 1. 为训练集定义的最终增强流水线 (精确复现你的等比例放大策略)
# --- Hybrid Mix 定义 ---
args_for_mix = KaggleConfig() # 获取类别数
cutmix = v2.CutMix(num_classes=args_for_mix.classnum,alpha=0.8)
mixup = v2.MixUp(num_classes=args_for_mix.classnum,alpha=0.8)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup], p =[0.7,0.3])

# --- Collate Function 定义 ---
def collate_fn(batch):
    """
    一个更健壮的collate_fn，用于处理包含额外信息（如文件名）的数据。
    """
    # 1. 先用PyTorch默认的方式将一批数据打包。
    # 对于我们返回 (文件名, 图像, 标签) 的数据集，这里会得到三个独立的批次。
    filenames, images, targets = default_collate(batch)
    
    # 2. 只把图像和标签传入CutMix/MixUp进行处理
    images, targets = cutmix_or_mixup(images, targets)
    
    # 3. 将原始的文件名列表和经过增强处理的图像、标签重新组合并返回
    return filenames, images, targets
    
def image_transform(mean,std):
    args = KaggleConfig()
    if args.aug_methon == 0:
        train_transform = A.Compose([
            # --- 核心策略：等比例放大 + 中心裁剪 ---
            # 这一步完全等效于 torchvision 的 transforms.Resize(int(224 * 1.14))
            # 它将图像的最短边缩放到 255 (即 224 * 1.14)，同时保持长宽比。
            A.SmallestMaxSize(max_size=int(224 * 1.14)), 
    
            # 这一步等效于 transforms.CenterCrop(224)
            A.CenterCrop(height=224, width=224),
    
             # --- 其他有效的增强技术 ---
            # 水平翻转
            A.HorizontalFlip(p=0.5), 
        
            # 轻微的几何变换 (位移、缩放、旋转)，对GPR图像的细微变化很有效
            A.ShiftScaleRotate(shift_limit=0.06, scale_limit=0.1, rotate_limit=5, p=0.7),

            # 模拟信号强度的变化
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    
            # 增强对噪声的抵抗力
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),

            # 随机遮挡，强迫模型关注全局特征
            #A.CoarseDropout(max_holes=8, max_height=20, max_width=20, p=0.5),
    
            # 标准化和类型转换
            A.Normalize(mean, std),
            ToTensorV2()
        ])
             # 2. 验证和测试集的预处理 (同样保持最短边缩放，保证处理逻辑一致性)
        val_test_transform = A.Compose([
            A.SmallestMaxSize(max_size=int(224 * 1.14)),
            A.CenterCrop(height=224, width=224),
            A.Normalize(mean,std),
            ToTensorV2()
        ])
    elif args.aug_methon == 1:
        train_transform = A.Compose([
        # --- 尺寸与几何变换 ---
        # 步骤 1: 使用 RandomResizedCrop 替代原来的固定缩放和中心裁剪。
        # 这是更现代、更强大的训练技巧，它随机地在图像中裁剪不同大小和长宽比的区域，然后缩放到224x224。
        # 这能让模型对目标的位置和大小变化更加鲁棒。
        A.LongestMaxSize(max_size=224, p=1.0),
        A.PadIfNeeded(min_height=224, min_width=224, border_mode=0, value=134, p=1.0),

        # 步骤 2: 水平翻转，对于GPR数据是安全且有效的增强。
        A.HorizontalFlip(p=0.5),

        # 步骤 3: 弹性变换，模拟地下介质不均匀导致的波形轻微扭曲，对GPR数据非常有效。
        A.ElasticTransform(p=0.5, alpha=20, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.ShiftScaleRotate(
            shift_limit=0.05,      # 轻微平移 (±5%)
            scale_limit=0.1,       # 轻微缩放 (±10%)
            rotate_limit=0,        # ❌ GPR数据不应旋转!
            border_mode=0,         # 黑色填充
            p=0.3                  # 降低概率,避免过度变换
        ),
        # --- 噪声与遮挡组合 ---
        # 步骤 4: 使用 SomeOf 组合器，每次以80%的概率从以下三种噪声/遮挡中随机选择两种来应用。
        # 这极大地增加了数据的多样性，避免模型对某种特定噪声产生过拟合。
        A.OneOf([
            # 选项1: 高斯噪声 - 模拟传感器的随机噪声
            A.GaussNoise(var_limit=(5.0, 30.0), p=1.0),  # ✅ 降低噪声强度: 10~50 -> 5~30
            
            # 选项2: 小范围随机遮挡 - 模拟数据缺失
            A.CoarseDropout(
                max_holes=4,           # ✅ 减少遮挡块数量: 8 -> 4
                max_height=15,         # ✅ 减小遮挡尺寸: 25 -> 15
                max_width=15, 
                min_holes=2,           # ✅ 减少最小遮挡数: 4 -> 2
                fill_value=0, 
                p=1.0
            ),
            
            # 选项3: 乘性噪声 - 模拟信号增益变化
            A.MultiplicativeNoise(multiplier=[0.9, 1.1], p=1.0),  # ✅ 收窄范围: 0.8~1.2 -> 0.9~1.1
        ], p=0.4),  # ✅ 提高应用概率: 0.3 -> 0.4 (因为现在只选一种,不会叠加

        # --- 信号强度与颜色变换 ---
        # 步骤 5: 随机调整亮度和对比度，模拟信号强度的变化。
        A.RandomBrightnessContrast(
            brightness_limit=0.15,  # ✅ 降低强度: 0.2 -> 0.15
            contrast_limit=0.15, 
            p=0.5                   # ✅ 降低概率: 0.7 -> 0.5
        ),

        # --- 标准化与格式转换 ---
        # 步骤 6: 标准化和转换为Tensor，这是任何模型都必需的步骤。
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
        ])
         # 2. 验证和测试集的预处理 (同样保持最短边缩放，保证处理逻辑一致性)
        val_test_transform = A.Compose([
            A.LongestMaxSize(max_size=224, p=1.0),
            A.PadIfNeeded(min_height=224, min_width=224, border_mode=0, value=134, p=1.0),
            A.Normalize(mean,std),
            ToTensorV2()
        ])
    return(train_transform ,val_test_transform)


In [7]:
# 定义全局类别映射和名称列表，方便后续使用
CLASS_MAP = {
    'Crack': 0, 'Loose': 1,
    'Mud Pumping': 2, 'Pipeline': 3, 'Redar': 4, 'Void': 5,
     'Water Abnormality': 6, 'stell_rib': 7 

}
CLASS_NAMES = [name for name, idx in sorted(CLASS_MAP.items(), key=lambda item: item[1])]

# 新的 DatasetImage 类，使用 albumentations
class DatasetImage(Dataset):
    def __init__(self, file_names, transform=None):
        self.file_names = file_names
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_file_name = self.file_names[idx]
        # 读取图像 (Albumentations 需要 RGB 格式)
        image = cv2.imread(img_file_name)
        if image is None: raise ValueError(f"无法读取图像: {img_file_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 应用增强
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        # 获取标签
        class_name = os.path.basename(os.path.dirname(img_file_name))
        cls = torch.tensor(CLASS_MAP[class_name], dtype=torch.long)
        
        return img_file_name, image, cls

# KaggleTestDataset 保持不变...
class KaggleTestDataset(Dataset):
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform
    def __len__(self):
        return len(self.file_paths)
    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return os.path.basename(img_path), img

In [8]:

def my_get_encoder(name, num_classes=1, pretrained=True, drop_rate=0.0): # 1. 新增 drop_rate 参数
    # 2. 在创建模型时，将 drop_rate 传递给 timm
    #    timm 会自动将其应用到模型的分类头
    encoder = timm.create_model(name, pretrained=pretrained, num_classes=num_classes, drop_rate=drop_rate)
    return encoder

class ClassificationModel(nn.Module):
    # 3. 在初始化时接收 drop_rate
    def __init__(self, encoder, classnum, pretrained=True, drop_rate=0.0): 
        super().__init__()
        # 4. 将 drop_rate 传递下去
        self.clf_model = my_get_encoder(encoder, num_classes=classnum, pretrained=pretrained, drop_rate=drop_rate)
        
    def forward(self, x):
        return self.clf_model(x)


In [9]:
def _calculate_metrics(metrics, mode, compute_cm):
    # --- 计算整体指标 ---
    precision = precision_score(metrics['all_targets'], metrics['all_preds'], average='weighted', zero_division=0)
    recall = recall_score(metrics['all_targets'], metrics['all_preds'], average='weighted', zero_division=0)
    f1 = f1_score(metrics['all_targets'], metrics['all_preds'], average='weighted', zero_division=0)
    f1_weighted = f1_score(metrics['all_targets'], metrics['all_preds'], average='weighted', zero_division=0)
    f1_macro = f1_score(metrics['all_targets'], metrics['all_preds'], average='macro', zero_division=0) # <--- 添加这一行
    # auc_macro, auc_weighted = 0.0, 0.0
    auc_macro, auc_weighted = 0.0, 0.0
    try:
        auc_macro = roc_auc_score(metrics['all_targets'], metrics['all_probs'], multi_class='ovr', average='macro')
        auc_weighted = roc_auc_score(metrics['all_targets'], metrics['all_probs'], multi_class='ovr', average='weighted')
    except Exception as e:
        logging.error(f"AUC计算失败: {e}")

    # --- 计算混淆矩阵和各类别指标 ---
    cm = confusion_matrix(metrics['all_targets'], metrics['all_preds']) if compute_cm and (mode in ['test', 'val']) else None
    per_class_recall = None
    per_class_precision = None
    if cm is not None:
        with np.errstate(divide='ignore', invalid='ignore'):
            # 各类别召回率(Recall) = 对角线元素 / 对应行的总和
            # 这也等同于该类别的准确率 (True Positive Rate)
            per_class_recall = cm.diagonal() / cm.sum(axis=1)
            per_class_recall[np.isnan(per_class_recall)] = 0

            # 各类别精确率(Precision) = 对角线元素 / 对应列的总和
            per_class_precision = cm.diagonal() / cm.sum(axis=0)
            per_class_precision[np.isnan(per_class_precision)] = 0

    return {
        'loss': metrics['loss'].avg, 'accuracy': metrics['accuracy'].avg,
        'precision': precision, 'recall': recall, 'f1': f1_weighted, 'f1_macro': f1_macro, # <--- 修改这一行
        'auc_macro': auc_macro, 'auc_weighted': auc_weighted,
        'confusion_matrix': cm,
        'per_class_recall': per_class_recall,
        'per_class_precision': per_class_precision
    }


def run_epoch(mode, model, data_loader, criterion, device, epoch, optimizer=None, writer=None, compute_cm=False, args=None):
    is_training = (mode == 'train')
    model.train() if is_training else model.eval()
    torch.set_grad_enabled(is_training)
    metrics = {'loss': AverageMeter("Loss", ".16f"), 'accuracy': AverageMeter("Acc", ".8f"), 'all_preds': [], 'all_targets': [], 'all_probs': []}
    pbar = tqdm(data_loader, desc=f"{mode.capitalize()} Epoch {epoch}")
    inference_start_time = time.time()   #记录时间
    
    # 🔥 修复: 如果没有传入 args,就创建一个默认的
    if args is None:
        args = KaggleConfig()
    
    for _, (file_name, images, target) in enumerate(pbar):
        images, target = images.to(device), target.to(device)
        output = model(images)
        loss = criterion(output, target)
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        probs = torch.softmax(output, dim=1).detach().cpu().numpy()
        preds = output.argmax(dim=1).cpu().numpy()
        # -- 核心修正：统一处理硬标签和软标签 --
        if args.use_hybrid_mix and target.ndim == 2:
            # 对于软标签，取 argmax 得到整数标签用于计算指标
            true_labels_for_metrics = target.argmax(dim=1)
        else:
            # 对于硬标签，直接使用
            true_labels_for_metrics = target

        # 使用修正后的 true_labels_for_metrics 来保存和计算
        metrics['all_probs'].extend(probs);
        metrics['all_preds'].extend(preds); 
        #metrics['all_targets'].extend(target.cpu().numpy())
        metrics['loss'].update(loss.item(), images.size(0))
        metrics['all_targets'].extend(true_labels_for_metrics.cpu().numpy())
        #metrics['accuracy'].update((preds == target.cpu().numpy()).mean(), images.size(0))
        # 将这一行:
        # metrics['accuracy'].update((preds == target.cpu().numpy()).mean(), images.size(0))
        # 替换为下面这个代码块:
        if args.use_hybrid_mix and target.ndim == 2:
            # 对于CutMix/MixUp生成的软标签(2D)，比较它们的argmax
            true_labels = target.argmax(dim=1)
        else:
            # 对于普通的硬标签(1D)
            true_labels = target
        metrics['accuracy'].update((preds == true_labels.cpu().numpy()).mean(), images.size(0))
        pbar.set_description(f"{mode:5} | Loss:{metrics['loss'].avg:.3f} Acc:{metrics['accuracy'].avg:.2%}")
    
    inference_end_time = time.time()
    final_metrics = _calculate_metrics(metrics, mode, compute_cm)

    # --- 写入TensorBoard日志 ---
    if writer:
        prefix = f"{mode}/"
        writer.add_scalar(f'{prefix}loss', final_metrics['loss'], epoch)
        writer.add_scalar(f'{prefix}accuracy', final_metrics['accuracy'], epoch)
        writer.add_scalar(f'{prefix}precision_weighted', final_metrics['precision'], epoch)
        writer.add_scalar(f'{prefix}recall_weighted', final_metrics['recall'], epoch)
        writer.add_scalar(f'{prefix}f1_weighted', final_metrics['f1'], epoch)
        writer.add_scalar(f'{prefix}f1_macro', final_metrics['f1_macro'], epoch) # <--- 添加这一行
        #writer.add_scalar(f'{prefix}auc_macro', final_metrics['auc_macro'], epoch)
        writer.add_scalar(f'{prefix}auc_macro', final_metrics['auc_macro'], epoch)
        writer.add_scalar(f'{prefix}auc_weighted', final_metrics['auc_weighted'], epoch)

        # 写入混淆矩阵图
        if final_metrics.get('confusion_matrix') is not None and mode in ['val', 'test']:
            writer.add_image(f'{prefix}confusion_matrix', cm_to_figure(final_metrics['confusion_matrix']), epoch)

        # (新增) 写入各类别指标
        if mode == 'test':
            
            report = classification_report(metrics['all_targets'], metrics['all_preds'], target_names=CLASS_NAMES, output_dict=True, zero_division=0)
            
            for class_name, class_metrics in report.items():
                if isinstance(class_metrics, dict): # 确保只处理类别行
                    writer.add_scalar(f'{prefix}{class_name}/precision', class_metrics['precision'], epoch)
                    writer.add_scalar(f'{prefix}{class_name}/recall', class_metrics['recall'], epoch)
                    writer.add_scalar(f'{prefix}{class_name}/f1-score', class_metrics['f1-score'], epoch)
                    writer.add_scalar(f'{prefix}{class_name}/support', class_metrics['support'], epoch)
            logging.info("已将每个类别的详细测试指标写入TensorBoard。")

    
    # 修正了这里的变量名：final_files -> final_metrics
    log_message = (f"{mode.capitalize()} Results - Loss: {final_metrics['loss']:.4f} | "
               f"Accuracy: {final_metrics['accuracy']:.4%} | "
               f"F1_weighted: {final_metrics['f1']:.4%} | "
               f"F1_macro: {final_metrics['f1_macro']:.4%}")
    if mode == 'test':
        total_inference_time = inference_end_time - inference_start_time
        num_samples = len(data_loader.dataset)
        fps = num_samples / total_inference_time
        log_message += f" | Inference Speed: {fps:.2f} FPS"
        logging.info(f"总推理时间 (Total Inference Time): {total_inference_time:.2f} seconds for {num_samples} images.")
        print("-" * 50)
        print(f"✅ 总推理时间 (Total Inference Time): {total_inference_time:.2f} seconds for {num_samples} images.")
        print(f"✅ 推理速度 (Inference Speed): {fps:.2f} FPS")
        print(log_message) # 也用 print 输出最终指标
        print("-" * 50)
    logging.info(log_message)
    
    # 🔥 修复: 从传入的 args 获取 run_id
    encoder_lr = getattr(args, 'run_id', None)
    save_metrics_to_excel(f"{mode}_metrics.xlsx", epoch, encoder_lr=encoder_lr, **final_metrics)
    return final_metrics


def predict_on_test(model, data_loader, device):
    model.eval()
    all_filenames, all_predictions = [], []
    with torch.no_grad():
        for filenames, images in tqdm(data_loader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_filenames.extend(filenames)
            all_predictions.extend(preds)
    return all_filenames, all_predictions


In [10]:
def build_scheduler(optimizer, phase, args, num_epochs_stage1=None):
    """
    phase: "stage1" 或 "stage2"
    num_epochs_stage1: 阶段一的训练轮数（仅用于计算 stage2 的 T_max）
    """
    if args.lr_scheduler == "cosine":
        if phase == "stage1":
            return torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=args.stage1_tmax, eta_min=args.min_lr
            )
        else:  # stage2
            # 根据总 epoch 自动计算阶段二的 T_max
            tmax_stage2 = args.num_epochs - (num_epochs_stage1 or 0)
            return torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=tmax_stage2, eta_min=args.min_lr
            )
    else:  # "plateau"
        # 注意：Plateau 的 step 需要传入验证损失，我们在训练循环里处理
        return torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",
            factor=args.plateau_factor,
            patience=args.plateau_patience,
            min_lr=args.min_lr,
            verbose=True
        )


In [ ]:
# =====================================================================
# 🆕 K折交叉验证训练脚本 (最小改动版 - 复用main函数)
# =====================================================================

from sklearn.model_selection import StratifiedKFold

def get_kfold_splits(file_names, n_folds=5, random_state=42):
    """创建K折交叉验证的数据分割"""
    labels = [CLASS_MAP[os.path.basename(os.path.dirname(p))] for p in file_names]
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    fold_splits = []
    for train_idx, val_idx in skf.split(file_names, labels):
        fold_splits.append((train_idx, val_idx))
    return fold_splits


def generate_dataset(train_file_names, val_file_names, test_file_names, batch_size, val_batch_size, test_batch_size, train_transform, val_test_transform):
    """生成数据加载器"""
    train_dataset = DatasetImage(train_file_names, transform=train_transform)
    valid_dataset = DatasetImage(val_file_names, transform=val_test_transform)
    test_dataset = DatasetImage(test_file_names, transform=val_test_transform)
    
    args = KaggleConfig()
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        num_workers=2, 
        shuffle=True, 
        drop_last=True,
        collate_fn=collate_fn if args.use_hybrid_mix else default_collate
    )
    valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, num_workers=2, shuffle=False, drop_last=False)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, num_workers=2, shuffle=False, drop_last=False)
    
    return train_loader, valid_loader, test_loader


def main(args, device, train_file_names, val_file_names, test_file_names):
    """主训练函数 (已修改为接收文件列表)"""
    
    print(f"找到 {len(train_file_names)} 个训练文件, {len(val_file_names)} 个验证文件, {len(test_file_names)} 个测试文件")
    
    # --- 计算均值标准差和数据增强 ---
    train_mean, train_std = mean_and_std(train_file_names)
    train_transform, val_test_transform = image_transform(mean=train_mean, std=train_std)
    
    train_loader, valid_loader, test_loader = generate_dataset(
        train_file_names, val_file_names, test_file_names,
        args.train_batch_size, args.val_batch_size, args.test_batch_size,
        train_transform=train_transform,
        val_test_transform=val_test_transform
    )
    
    # --- 计算类别权重 ---
    print("正在计算类别权重以处理数据不平衡问题...")
    train_labels = [CLASS_MAP[os.path.basename(os.path.dirname(p))] for p in train_file_names]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    print(f"计算出的类别权重: {class_weights.cpu().numpy()}")
    
    # --- 初始化模型 ---
    print(f"\n正在初始化模型: {args.encoder}")
    model = ClassificationModel(encoder=args.encoder, classnum=args.classnum, pretrained=True, drop_rate=args.dropout_rate).to(device)
    
    model_for_profiling = deepcopy(model)
    model_for_profiling.eval()
    num_params = count_parameters(model)
    print("=" * 60)
    print(f"✅ 模型参数量 (Parameters): {num_params / 1e6:.2f} M")
    logging.info("==================== MODEL EFFICIENCY ====================")
    logging.info(f"模型参数量 (Parameters): {num_params / 1e6:.2f} M")
    dummy_input = torch.randn(1, 3, 224, 224).to(device)
    flops, params = profile(model_for_profiling, inputs=(dummy_input,))
    print(f"✅ FLOPs: {flops / 1e9:.2f} G")
    print("=" * 60)
    logging.info(f"FLOPs: {flops / 1e9:.2f} G")
    logging.info("==========================================================")
    del model_for_profiling
    
    # --- 损失函数 ---
    if args.use_hybrid_mix:
        print("正在使用 nn.CrossEntropyLoss (因为启用了 CutMix/MixUp)")
        criterion = nn.CrossEntropyLoss(label_smoothing=args.label_smoothing, weight=class_weights)
    else:
        print("正在使用 FocalLoss")
        criterion = FocalLoss(alpha=class_weights, gamma=args.focal_loss_gamma)
    
    writer = SummaryWriter(log_dir=args.save_path)
    training_start_time = time.time()
    
    # =====================================================================================
    # 阶段一: 冻结主干网络，只训练分类头
    # =====================================================================================
    print("\n" + "="*50)
    print("🔥 阶段一: 冻结主干网络，只训练分类头")
    print("="*50)
    
    for param in model.clf_model.parameters():
        param.requires_grad = False
    
    if hasattr(model.clf_model, 'head'):
        for param in model.clf_model.head.parameters():
            param.requires_grad = True
        print("模型主干已冻结，只训练最后的 'head' 层。")
    elif hasattr(model.clf_model, 'classifier'):
        for param in model.clf_model.classifier.parameters():
            param.requires_grad = True
        print("模型主干已冻结，只训练最后的 'classifier' 层。")
    elif hasattr(model.clf_model, 'fc'):
        for param in model.clf_model.fc.parameters():
            param.requires_grad = True
        print("模型主干已冻结，只训练最后的 'fc' 层。")
    else:
        raise AttributeError("无法在模型中找到 'head', 'classifier' 或 'fc' 层进行解冻。")
    
    num_epochs_stage1 = 15
    optimizer_stage1 = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.LR_head, weight_decay=args.WE_dec)
    scheduler_stage1 = build_scheduler(optimizer_stage1, phase="stage1", args=args)
    
    best_acc_stage1 = 0
    for epoch in range(1, num_epochs_stage1 + 1):
        logging.info(f"阶段一 - Epoch {epoch}/{num_epochs_stage1}")
        run_epoch('train', model, train_loader, criterion, device, epoch, optimizer=optimizer_stage1, writer=writer, args=args)
        val_metrics = run_epoch('val', model, valid_loader, criterion, device, epoch, writer=writer, args=args)
        
        if args.lr_scheduler == "cosine":
            scheduler_stage1.step()
        else:
            scheduler_stage1.step(val_metrics['loss'])
        
        if val_metrics['accuracy'] >= best_acc_stage1:
            best_acc_stage1 = val_metrics['accuracy']
            logging.info(f"阶段一新高! 验证集准确率: {best_acc_stage1:.4%}. 保存模型...")
            torch.save(model.state_dict(), os.path.join(args.save_path, 'best_model_stage1.pth'))
    
    # =====================================================================================
    # 阶段二: 解冻所有层，进行全局微调
    # =====================================================================================
    print("\n" + "="*50)
    print("🔥 阶段二: 解冻所有层，进行全局微调")
    print("="*50)
    print("加载阶段一的最佳模型权重...")
    model.load_state_dict(torch.load(os.path.join(args.save_path, 'best_model_stage1.pth')))
    for param in model.parameters():
        param.requires_grad = True
    print("所有层已解冻。")
    
    optimizer_stage2 = Adam(model.parameters(), lr=args.LR_clf, weight_decay=args.WE_dec)
    # 🔥 修复: 统一使用 build_scheduler，保持两阶段调度策略一致
    scheduler_stage2 = build_scheduler(optimizer_stage2, phase="stage2", args=args, num_epochs_stage1=num_epochs_stage1)
    
    best_acc = 0
    best_epoch = 0
    early_stopping_patience = 35
    epochs_no_improve = 0
    
    for epoch in range(num_epochs_stage1 + 1, args.num_epochs + 1):
        logging.info(f"阶段二 - Epoch {epoch}/{args.num_epochs}")
        
        run_epoch('train', model, train_loader, criterion, device, epoch, optimizer=optimizer_stage2, writer=writer, args=args)
        val_metrics = run_epoch('val', model, valid_loader, criterion, device, epoch, writer=writer, args=args)
        
        # 🔥 修复: 根据配置的调度器类型调用不同的 step 方法
        if args.lr_scheduler == "cosine":
            scheduler_stage2.step()
        else:
            scheduler_stage2.step(val_metrics['loss'])
        
        if val_metrics['accuracy'] >= best_acc:
            best_acc = val_metrics['accuracy']
            best_epoch = epoch
            epochs_no_improve = 0
            logging.info(f"全局新高! 验证集准确率: {best_acc:.4%}. 保存模型...")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
            }, os.path.join(args.save_path, 'best_model.pth'))
        else:
            epochs_no_improve += 1
            logging.info(f"验证集准确率未提升，当前无改善轮数: {epochs_no_improve}/{early_stopping_patience}")
        
        if epochs_no_improve >= early_stopping_patience:
            print(f'\n')
            logging.info(f"触发提前停止！连续 {early_stopping_patience} 个 epochs 验证集准确率未提升。")
            logging.info(f"最佳epoch: {best_epoch}, 最佳验证准确率: {best_acc:.4%}")
            break
    
    training_end_time = time.time()
    total_training_time = training_end_time - training_start_time
    logging.info(f"总训练时间 (Total Training Time): {total_training_time/3600:.2f} hours")
    
    # --- 最终测试评估 ---
    print("\n" + "="*50)
    print(f"🔥 评估全局最佳模型 (来自 epoch {best_epoch}，验证集准确率 {best_acc:.4%})")
    
    best_model_path = os.path.join(args.save_path, 'best_model.pth')
    test_metrics = None
    if os.path.exists(best_model_path):
        final_test_model = ClassificationModel(encoder=args.encoder, classnum=args.classnum, pretrained=False).to(device)
        checkpoint = torch.load(best_model_path)
        final_test_model.load_state_dict(checkpoint['model_state_dict'])
        
        test_metrics = run_epoch('test', final_test_model, test_loader, criterion, device,
                                 epoch=checkpoint['epoch'], writer=writer, compute_cm=True, args=args)
        
        if test_metrics.get('confusion_matrix') is not None:
            cm = test_metrics['confusion_matrix']
            print("\nFinal Test Confusion Matrix:")
            for i in range(cm.shape[0]):
                row_str = " | ".join([f"{count:4d}" for count in cm[i]])
                print(f"Class {i:<2} | {row_str}")
    
    print("="*50 + "\n")
    torch.save(model.state_dict(), os.path.join(args.save_path, 'final_model.pth'))
    
    # --- 绘制历史曲线 ---
    print("\n正在生成训练历史曲线图...")
    try:
        train_metrics_df = pd.read_excel(os.path.join(args.save_path, 'train_metrics.xlsx'))
        val_metrics_df = pd.read_excel(os.path.join(args.save_path, 'val_metrics.xlsx'))
        history = {
            'train_loss': train_metrics_df['Loss'].tolist(),
            'train_acc': train_metrics_df['Accuracy'].tolist(),
            'val_loss': val_metrics_df['Loss'].tolist(),
            'val_acc': val_metrics_df['Accuracy'].tolist()
        }
        plot_history_curves(history, args.save_path)
    except FileNotFoundError:
        print("Warning: 未找到 train_metrics.xlsx 或 val_metrics.xlsx。跳过绘图。")
    except Exception as e:
        print(f"绘图时发生错误: {e}")
    
    return {
        'best_epoch': best_epoch,
        'best_val_acc': best_acc,
        'test_metrics': test_metrics
    }


# =====================================================================
# 主训练循环 - K折交叉验证
# =====================================================================

if __name__ == '__main__':
    # --- 全局设置 ---
    args = KaggleConfig()
    set_seed(42)
    device = torch.device(f'cuda:{args.cuda_no}' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    os.makedirs(args.save_path, exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f"🔥 开始 {args.n_folds} 折交叉验证训练")
    print(f"模型: {args.encoder} | 学习率: {args.LR_clf}")
    print(f"{'='*70}\n")
    
    # --- 获取所有训练+验证文件（合并用于K折分割）---
    train_files = glob.glob(os.path.join(args.train_path, '*', '*.[jp][pn]g'))
    val_files = glob.glob(os.path.join(args.val_path, '*', '*.[jp][pn]g'))
    all_train_val_files = train_files + val_files
    test_file_names = glob.glob(os.path.join(args.test_path, '*', '*.[jp][pn]g'))
    
    print(f"总训练+验证文件数: {len(all_train_val_files)}")
    print(f"测试文件数: {len(test_file_names)}")
    
    # --- 创建K折分割 ---
    fold_splits = get_kfold_splits(all_train_val_files, n_folds=args.n_folds, random_state=42)
    
    # --- 存储所有fold的结果 ---
    all_fold_results = []
    
    # --- 训练每个fold (🆕 支持指定fold范围) ---
    for temp_idx, (train_indices, val_indices) in enumerate(fold_splits[args.start_fold:args.end_fold]):
        fold_idx = temp_idx + args.start_fold  # 调整为实际的fold索引
        print(f"\n{'='*70}")
        print(f"🔥 开始训练 Fold {fold_idx + 1}/{args.n_folds}")
        print(f"{'='*70}\n")
        
        # 根据索引获取文件名
        fold_train_files = [all_train_val_files[i] for i in train_indices]
        fold_val_files = [all_train_val_files[i] for i in val_indices]
        
        # 🔥 设置当前fold的保存路径和run_id
        fold_save_path = os.path.join(args.save_path, f'fold_{fold_idx}')
        os.makedirs(fold_save_path, exist_ok=True)
        original_save_path = args.save_path
        args.save_path = fold_save_path
        args.current_fold = fold_idx
        args.run_id = f"{args.encoder}_lr_{args.LR_clf}_fold{fold_idx}"
        
        # 训练当前fold
        fold_result = main(args, device, fold_train_files, fold_val_files, test_file_names)
        fold_result['fold'] = fold_idx
        all_fold_results.append(fold_result)
        
        # 恢复原始保存路径
        args.save_path = original_save_path
        
        print(f"\n✅ Fold {fold_idx + 1} 训练完成!")
        print(f"   最佳epoch: {fold_result['best_epoch']}")
        print(f"   最佳验证准确率: {fold_result['best_val_acc']:.4%}")
        if fold_result['test_metrics']:
            print(f"   测试准确率: {fold_result['test_metrics']['accuracy']:.4%}")
            print(f"   测试F1 (macro): {fold_result['test_metrics']['f1_macro']:.4%}")
    
    # --- 汇总所有fold的结果 ---
    print(f"\n{'='*70}")
    print(f"🎉 所有 {args.end_fold - args.start_fold} 折训练完成!")
    print(f"{'='*70}\n")
    
    avg_val_acc = np.mean([r['best_val_acc'] for r in all_fold_results])
    std_val_acc = np.std([r['best_val_acc'] for r in all_fold_results])
    
    avg_test_acc = np.mean([r['test_metrics']['accuracy'] for r in all_fold_results if r['test_metrics']])
    std_test_acc = np.std([r['test_metrics']['accuracy'] for r in all_fold_results if r['test_metrics']])
    
    avg_test_f1 = np.mean([r['test_metrics']['f1_macro'] for r in all_fold_results if r['test_metrics']])
    std_test_f1 = np.std([r['test_metrics']['f1_macro'] for r in all_fold_results if r['test_metrics']])
    
    print("📊 K折交叉验证汇总结果:")
    print(f"   验证准确率: {avg_val_acc:.4%} ± {std_val_acc:.4%}")
    print(f"   测试准确率: {avg_test_acc:.4%} ± {std_test_acc:.4%}")
    print(f"   测试F1 (macro): {avg_test_f1:.4%} ± {std_test_f1:.4%}")
    
    # --- 保存汇总结果 ---
    summary_df = pd.DataFrame(all_fold_results)
    summary_path = os.path.join(args.save_path, 'kfold_summary.csv')
    summary_df.to_csv(summary_path, index=False)
    print(f"\n✅ K折汇总结果已保存至: {summary_path}")
